In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import *
import numpy as np

spark = (
    SparkSession.builder.appName("ADS project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/05 02:45:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/05 02:45:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/05 02:45:05 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/05 02:45:05 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
df = spark.read.parquet('../data/curated/merchant_consumer_abs')

In [3]:
df

22/10/05 02:45:17 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


postcode,total_earners,median_age,income_sum,income_median,income_mean,2021_population,km2,persons/km2,merchant_name,revenue_level,user_id,order_datetime,products,take_rate,category,dollar_value,order_year,order_month,order_day,consumer,consumer_address,consumer_state,consumer_postcode,consumer_gender,fraud_group,__index_level_0__
0800,5631.999973601934,33.00000004463053,4.20609029345557E8,57789.00008173299,74682.00012457666,7678.999968193022,3.199999994700948,2419.9999940679677,Elit Sed Consequa...,a,10413,2022-04-27,artist supply and...,5.89,art and gifts,375.16773164703153,2022,4,27,Kelly Clayton,4211 Rodney Tunne...,NT,0800,Female,0,0
0800,5631.999973601934,33.00000004463053,4.20609029345557E8,57789.00008173299,74682.00012457666,7678.999968193022,3.199999994700948,2419.9999940679677,Elit Sed Consequa...,a,22246,2022-10-07,artist supply and...,5.89,art and gifts,617.8791313462011,2022,10,7,Corey Estrada,1703 Boyd Shore,NT,0800,Male,0,1
0800,5631.999973601934,33.00000004463053,4.20609029345557E8,57789.00008173299,74682.00012457666,7678.999968193022,3.199999994700948,2419.9999940679677,Elit Sed Consequa...,a,2681,2022-02-28,artist supply and...,5.89,art and gifts,766.5077067424303,2022,2,28,Nicole Bishop,467 Robert Island...,NT,0800,Female,0,2
0800,5631.999973601934,33.00000004463053,4.20609029345557E8,57789.00008173299,74682.00012457666,7678.999968193022,3.199999994700948,2419.9999940679677,Malesuada Vel Ltd,b,13454,2021-07-24,books periodicals...,3.56,books and music,258.86595899218753,2021,7,24,Natalie Herrera,88798 Saunders Hi...,NT,0800,Female,0,3
0800,5631.999973601934,33.00000004463053,4.20609029345557E8,57789.00008173299,74682.00012457666,7678.999968193022,3.199999994700948,2419.9999940679677,Varius Orci Insti...,a,10146,2021-08-08,tent and awning s...,6.3,outdoors,4.159037931172075,2021,8,8,Shannon Mann,00817 Owens Circle,NT,0800,Female,0,4
0800,5631.999973601934,33.00000004463053,4.20609029345557E8,57789.00008173299,74682.00012457666,7678.999968193022,3.199999994700948,2419.9999940679677,Volutpat Nulla In...,a,23093,2022-10-11,furniture home fu...,6.59,home and technology,8.26105405068297,2022,10,11,Robert Walker,646 Allison Ranch...,NT,0800,Male,0,5
0800,5631.999973601934,33.00000004463053,4.20609029345557E8,57789.00008173299,74682.00012457666,7678.999968193022,3.199999994700948,2419.9999940679677,Mollis Integer Co...,b,13454,2022-05-02,digital goods boo...,3.96,books and music,83.63154755239155,2022,5,2,Natalie Herrera,88798 Saunders Hi...,NT,0800,Female,0,6
0800,5631.999973601934,33.00000004463053,4.20609029345557E8,57789.00008173299,74682.00012457666,7678.999968193022,3.199999994700948,2419.9999940679677,Elit Sed Consequa...,a,13454,2021-04-08,artist supply and...,5.89,art and gifts,87.00495112586256,2021,4,8,Natalie Herrera,88798 Saunders Hi...,NT,0800,Female,0,7
0800,5631.999973601934,33.00000004463053,4.20609029345557E8,57789.00008173299,74682.00012457666,7678.999968193022,3.199999994700948,2419.9999940679677,Eros Limited,c,8092,2021-12-18,digital goods boo...,2.52,books and music,8.26187754879106,2021,12,18,Rebecca Owens,92579 Keith Neck ...,NT,0800,Female,0,8
0800,5631.999973601934,33.00000004463053,4.20609029345557E8,57789.00008173299,74682.00012457666,7678.999968193022,3.199999994700948,2419.9999940679677,Hendrerit A Corpo...,a,11203,2022-07-14,watch clock and j...,6.64,fashion and acces...,100.80643267043831,2022,7,14,John Rodriguez,563 Wallace Rue,NT,0800,Male,0,9


<h2>1. Split dataset</h2>

In [4]:
import time
import datetime
date = "2022-08-31"
date_time = datetime.datetime.strptime(date, '%Y-%m-%d')

In [5]:
df_train = df.where((col("order_datetime") <= date_time.date()))
df_train.count()

9873203

In [6]:
df_test = df.where((col("order_datetime") > date_time.date()))
df_test.count()

1214722

<h2>2. Aggregate dataset</h2>

In [7]:
df_train_agg = df_train.groupBy(["merchant_name", "order_year", "order_month"]).agg(
    sum("dollar_value").alias("total_revenue"), # response variable

    max("take_rate").alias("take_rate"),
    avg("total_earners").alias("total_earners"),
    avg("median_age").alias("median_age"),
    avg("income_median").alias("income_median"),
    avg("persons/km2").alias("population_density"),
    max("category").alias("category"),
    max("revenue_level").alias("revenue_level")
    )

df_test_agg = df_test.groupBy(["merchant_name", "order_year", "order_month"]).agg(
    sum("dollar_value").alias("total_revenue"), # response variable

    max("take_rate").alias("take_rate"),
    avg("total_earners").alias("total_earners"),
    avg("median_age").alias("median_age"),
    avg("income_median").alias("income_median"),
    avg("persons/km2").alias("population_density"),
    max("category").alias("category"),
    max("revenue_level").alias("revenue_level")
    )

In [8]:
df_train_agg = df_train_agg.withColumn("revenue_taken", F.col("total_revenue") * (F.col("take_rate") / 100))
df_test_agg = df_test_agg.withColumn("revenue_taken", F.col("total_revenue") * (F.col("take_rate") / 100))

In [9]:
df_train_agg = df_train_agg.drop("total_revenue", "take_rate")
df_test_agg = df_test_agg.drop("total_revenue", "take_rate")

In [10]:
df_train_agg

merchant_name,order_year,order_month,total_earners,median_age,income_median,population_density,category,revenue_level,revenue_taken
A Aliquet Ltd,2021,3,4980.623604961794,45.955320262557386,44348.234836395015,446.43100140347246,home and technology,b,349.5484134530372
A Aliquet Ltd,2021,4,5354.985464077088,44.957488804012904,47087.020093628846,661.3048628221844,home and technology,b,338.07536243758386
A Aliquet Ltd,2021,7,4388.982031619088,44.56853448053012,44286.808317322975,322.56914352472995,home and technology,b,475.18502622308233
A Aliquet Ltd,2021,8,5281.06502903092,41.808178897353294,44432.17706152081,674.1575473010176,home and technology,b,375.8536740741011
A Aliquet Ltd,2021,12,4218.068045625177,44.00092016580645,45702.003770359916,633.7586224746047,home and technology,b,486.47663958043245
A Aliquet Ltd,2022,6,4907.6821031588515,44.52282551974433,46859.1278088761,506.7066383636061,home and technology,b,383.8019227224976
A Aliquet Ltd,2022,8,4587.962567801989,43.09698805802131,45936.13593995373,564.2479724970453,home and technology,b,537.5835117882583
A Arcu Industries,2021,2,3464.1233178609377,46.506422301920416,51956.04631148328,12.921544307761414,outdoors,c,21.744568393051882
A Arcu Industries,2021,12,4631.073949176494,43.00178251688292,45797.775562465315,590.2452906166948,outdoors,c,1582.2315307816057
A Arcu Industries,2022,5,4953.264665124492,41.985087824662386,46570.62536428412,839.7026423642426,outdoors,c,1203.3213130052611


In [11]:
df_train_agg.printSchema()

root
 |-- merchant_name: string (nullable = true)
 |-- order_year: integer (nullable = true)
 |-- order_month: integer (nullable = true)
 |-- total_earners: double (nullable = true)
 |-- median_age: double (nullable = true)
 |-- income_median: double (nullable = true)
 |-- population_density: double (nullable = true)
 |-- category: string (nullable = true)
 |-- revenue_level: string (nullable = true)
 |-- revenue_taken: double (nullable = true)

